In [1]:
# Import numpy and TF backend
import numpy as np
import os
import tensorflow as tf
tf.python.control_flow_ops = tf

# Import Keras 
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers import Cropping2D, Lambda
from keras import backend as K

# Import cv
import cv2
import csv
import sklearn



Using TensorFlow backend.


In [2]:
# Global variables
dataFolders = []
homeFolder = os.getenv("HOME")
dataFolders.append(homeFolder+'/Data/')
dataFolders.append(homeFolder+'/Desktop/data/')

In [3]:
# Functions


# get ImageMat for the given file location in the Data log
def process_image(folder, imageField):
    imagePath = imageField
    fileName = imagePath.split('/')[-1]
    fileNamewithFolder = folder +'/IMG/' + fileName
    imageMat = cv2.imread(fileNamewithFolder)
    
    return imageMat

def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        sklearn.utils.shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            msm_init_count = len(angles)
            for batch_sample in batch_samples:
                imageMat = cv2.imread(batch_sample[0])
                steering_center = float(batch_sample[3])
                images.append(imageMat)
                angles.append(steering_center)
                # create adjusted steering measurements for the side camera images
                correction = 0.25 # this is a parameter to tune
                steering_left = steering_center + correction
                steering_right = steering_center - correction

                # read in images from center, left and right cameras
                imageMat =  cv2.imread(batch_sample[1])
                images.append(imageMat)
                angles.append(steering_left)

                imageMat =  cv2.imread(batch_sample[2])
                images.append(imageMat)
                angles.append(steering_right)
            
            # Find the existing length of images
            list_len = len(angles)
            for i in range(msm_init_count, list_len):
                # Now add the image flip side to reduce the left bias
                image_flipped = np.fliplr(images[i])
                msmt_flipped = -1 * angles[i]
                images.append(image_flipped)
                angles.append(msmt_flipped)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)

In [4]:
lines = []
for dataFolder in dataFolders:

    with open(dataFolder+'driving_log.csv') as csvfile:
        reader = csv.reader(csvfile)
        for line in reader:
            lines.append(line)
        print(len(lines))
        print (lines[0][1])

from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(lines, test_size=0.2)
'''
    # Traverse thru the lines and extract images (X values) and steering value (y value)
    msm_init_count = len(msmts)
    for line in lines:
        # First element is the image
        imageMat = cv2.imread(line[0])
        images.append(imageMat)
        # 4th element is the steering info
        steering_center = float(line[3])
        msmts.append(steering_center)

        # create adjusted steering measurements for the side camera images
        correction = 0.25 # this is a parameter to tune
        steering_left = steering_center + correction
        steering_right = steering_center - correction

        # read in images from center, left and right cameras
        imageMat = cv2.imread(line[1])
        images.append(imageMat)
        msmts.append(steering_left)

        imageMat =  cv2.imread(line[2])
        images.append(imageMat)
        msmts.append(steering_right)

    # Find the existing length of images
    list_len = len(msmts)
    for i in range(msm_init_count, list_len):

        # Now add the image flip side to reduce the left bias
        image_flipped = np.fliplr(images[i])
        msmt_flipped = -1 * msmts[i]
        images.append(image_flipped)
        msmts.append(msmt_flipped)

print(len(msmts))
'''

4706
/home/ramesh/Data/IMG/left_2017_07_12_06_34_58_813.jpg
15042
/home/ramesh/Data/IMG/left_2017_07_12_06_34_58_813.jpg


'\n    # Traverse thru the lines and extract images (X values) and steering value (y value)\n    msm_init_count = len(msmts)\n    for line in lines:\n        # First element is the image\n        imageMat = cv2.imread(line[0])\n        images.append(imageMat)\n        # 4th element is the steering info\n        steering_center = float(line[3])\n        msmts.append(steering_center)\n\n        # create adjusted steering measurements for the side camera images\n        correction = 0.25 # this is a parameter to tune\n        steering_left = steering_center + correction\n        steering_right = steering_center - correction\n\n        # read in images from center, left and right cameras\n        imageMat = cv2.imread(line[1])\n        images.append(imageMat)\n        msmts.append(steering_left)\n\n        imageMat =  cv2.imread(line[2])\n        images.append(imageMat)\n        msmts.append(steering_right)\n\n    # Find the existing length of images\n    list_len = len(msmts)\n    for i i

In [5]:
# Set the traing data
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)
#print(len(train_generator))

In [6]:
# TODO: Build the Fully Connected Neural Network in Keras Here
model = Sequential()

model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((50,20), (0,0))))
model.add(Convolution2D(24, 5, 5,
                activation='relu',
                border_mode='valid'))
model.add(MaxPooling2D((2, 2)))
model.add(Convolution2D(32, 5, 5,
                activation='relu',
                border_mode='valid'))
model.add(MaxPooling2D((2, 2)))
model.add(Convolution2D(48, 5, 5,
                activation='relu',
                border_mode='valid'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.5))

model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(Dropout(0.25))

model.add(Flatten())

#2nd Layer - Add a fully connected layer
model.add(Dense(128))
#3rd Layer - Add a ReLU activation layer
#model.add(Activation('relu'))
#model.add(Dropout(0.5))
model.add(Dense(42))
model.add(Dense(10))
#4th Layer - Add a fully connected layer - one node at the end to predict steering (basically regression)
model.add(Dense(1))

In [7]:
# Times the the augmented data is generated for each line 
nAugmentedFactor = 6
# Train the model
model.compile(loss='mse', optimizer='adam')
#history = model.fit(X_train, Y_train, nb_epoch=5, validation_split=0.2, shuffle=True)
history = model.fit_generator(train_generator, samples_per_epoch= 
            nAugmentedFactor*len(train_samples), validation_data=validation_generator, 
            nb_val_samples=nAugmentedFactor*len(validation_samples), nb_epoch=6)

model.save('bc_model_jul13.h5')

Epoch 1/3
72198/72198 [==============================] - 123s - loss: 0.0832 - val_loss: 0.0773
Epoch 2/3
72198/72198 [==============================] - 111s - loss: 0.0738 - val_loss: 0.0757
Epoch 3/3
72198/72198 [==============================] - 114s - loss: 0.0715 - val_loss: 0.0700
